# Tugas Individu Teknologi Basis Data
- Nama: Muhammad Haerul
- NIM: H071201033
- Kelas: A 

## Create Engine

In [1]:
from sqlalchemy import create_engine
engine = create_engine('mysql://root@localhost/classicmodels', echo=False)

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
Base.metadata.create_all(engine)

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind = engine)
session = Session()

## Create Models

In [2]:
from sqlalchemy import Column, DECIMAL, Date, ForeignKey, String, Text
from sqlalchemy.dialects.mysql import INTEGER, MEDIUMBLOB, MEDIUMTEXT, SMALLINT
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata

class Office(Base):
    __tablename__ = 'offices'

    officeCode = Column(String(10), primary_key=True)
    city = Column(String(50), nullable=False)
    phone = Column(String(50), nullable=False)
    addressLine1 = Column(String(50), nullable=False)
    addressLine2 = Column(String(50))
    state = Column(String(50))
    country = Column(String(50), nullable=False)
    postalCode = Column(String(15), nullable=False)
    territory = Column(String(10), nullable=False)


class Productline(Base):
    __tablename__ = 'productlines'

    productLine = Column(String(50), primary_key=True)
    textDescription = Column(String(4000))
    htmlDescription = Column(MEDIUMTEXT)
    image = Column(MEDIUMBLOB)


class Employee(Base):
    __tablename__ = 'employees'

    employeeNumber = Column(INTEGER(11), primary_key=True)
    lastName = Column(String(50), nullable=False)
    firstName = Column(String(50), nullable=False)
    extension = Column(String(10), nullable=False)
    email = Column(String(100), nullable=False)
    officeCode = Column(ForeignKey('offices.officeCode'), nullable=False, index=True)
    reportsTo = Column(ForeignKey('employees.employeeNumber'), index=True)
    jobTitle = Column(String(50), nullable=False)

    office = relationship('Office')
    parent = relationship('Employee', remote_side=[employeeNumber])


class Product(Base):
    __tablename__ = 'products'

    productCode = Column(String(15), primary_key=True)
    productName = Column(String(70), nullable=False)
    productLine = Column(ForeignKey('productlines.productLine'), nullable=False, index=True)
    productScale = Column(String(10), nullable=False)
    productVendor = Column(String(50), nullable=False)
    productDescription = Column(Text, nullable=False)
    quantityInStock = Column(SMALLINT(6), nullable=False)
    buyPrice = Column(DECIMAL(10, 2), nullable=False)
    MSRP = Column(DECIMAL(10, 2), nullable=False)

    productline = relationship('Productline')


class Customer(Base):
    __tablename__ = 'customers'

    customerNumber = Column(INTEGER(11), primary_key=True)
    customerName = Column(String(50), nullable=False)
    contactLastName = Column(String(50), nullable=False)
    contactFirstName = Column(String(50), nullable=False)
    phone = Column(String(50), nullable=False)
    addressLine1 = Column(String(50), nullable=False)
    addressLine2 = Column(String(50))
    city = Column(String(50), nullable=False)
    state = Column(String(50))
    postalCode = Column(String(15))
    country = Column(String(50), nullable=False)
    salesRepEmployeeNumber = Column(ForeignKey('employees.employeeNumber'), index=True)
    creditLimit = Column(DECIMAL(10, 2))

    employee = relationship('Employee')


class Order(Base):
    __tablename__ = 'orders'

    orderNumber = Column(INTEGER(11), primary_key=True)
    orderDate = Column(Date, nullable=False)
    requiredDate = Column(Date, nullable=False)
    shippedDate = Column(Date)
    status = Column(String(15), nullable=False)
    comments = Column(Text)
    customerNumber = Column(ForeignKey('customers.customerNumber'), nullable=False, index=True)

    customer = relationship('Customer')


class Payment(Base):
    __tablename__ = 'payments'

    customerNumber = Column(ForeignKey('customers.customerNumber'), primary_key=True, nullable=False)
    checkNumber = Column(String(50), primary_key=True, nullable=False)
    paymentDate = Column(Date, nullable=False)
    amount = Column(DECIMAL(10, 2), nullable=False)

    customer = relationship('Customer')


class Orderdetail(Base):
    __tablename__ = 'orderdetails'

    orderNumber = Column(ForeignKey('orders.orderNumber'), primary_key=True, nullable=False)
    productCode = Column(ForeignKey('products.productCode'), primary_key=True, nullable=False, index=True)
    quantityOrdered = Column(INTEGER(11), nullable=False)
    priceEach = Column(DECIMAL(10, 2), nullable=False)
    orderLineNumber = Column(SMALLINT(6), nullable=False)

    order = relationship('Order')
    product = relationship('Product')

## Define and Import Helper Functions

In [25]:
import pandas as pd
from sqlalchemy.sql import func
from sqlalchemy import or_
from sqlalchemy import desc

def to_df(query):
    df = pd.read_sql(query.statement, engine)
    return df

def show_df(query):
    df = pd.read_sql(query.statement, engine)
    return df

## Praktikum 3

1. Tampilkan semua kolom dari tabel offices yang berasal dari
kota ‘San Francisco’.

In [6]:
num1 = session.query(Office).filter(Office.city == 'San Francisco')
df1 = to_df(num1)
df1

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA


2 Tampilkan semua kolom dari tabel orderdetails, di mana
quantityOrdered lebih besar dari 70, diurutkan secara menaik
berdasarkan orderLineNumber.

In [7]:
od = Orderdetail
num2 = session.query(od).filter(od.quantityOrdered >= 70).order_by(od.orderLineNumber)
df2 = to_df(num2)
df2

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10407,S18_1749,76,141.10,2
1,10405,S24_3856,76,127.79,3
2,10404,S12_3990,77,67.03,4
3,10405,S12_4675,97,115.16,5
4,10407,S24_2766,76,81.78,6
5,10404,S18_3278,90,67.54,6
6,10419,S24_3856,70,112.34,8
7,10401,S700_3167,77,73.60,9
8,10412,S24_2300,70,109.90,10
9,10401,S700_2466,85,98.72,10


3. Tampilkan productLine tanpa ada value yang sama pada tabel
products

In [8]:
s = session
p = Product

num3 = s.query(p.productLine).distinct()

df3 = to_df(num3)
df3

,productLine
0,Classic Cars
1,Motorcycles
2,Planes
3,Ships
4,Trains
5,Trucks and Buses
6,Vintage Cars


4. Tampilkan customerNumber dan customerName pada tabel
customers dengan customerNumber di kisaran 100-150. Jangan
lupa berikan alias pada kolom keluaran.

In [9]:
c = Customer

num4 = s.query(c.customerNumber, c.customerName).filter(100 <= c.customerNumber, c.customerNumber <= 150)

df4 = to_df(num4)
df4

,customerNumber,customerName
0,103,Atelier graphique
1,112,Signal Gift Stores
2,114,"Australian Collectors, Co."
3,119,La Rochelle Gifts
4,121,Baane Mini Imports
5,124,Mini Gifts Distributors Ltd.
6,125,Havel & Zbyszek Co
7,128,"Blauer See Auto, Co."
8,129,Mini Wheels Co.
9,131,Land of Toys Inc.


5. Berikan Data customers yang bukan dari USA yang tidak bisa
lagi menarik uang dari rekeningnya. Urutkan berdasarkan
customerName secara menaik lalu berikan data dari index
(inklusif) 10 sampai 19.

In [43]:
c = Customer

num5 = s.query(c).filter(c.country != 'USA', c.creditLimit != 0).order_by(c.customerName).offset(10).limit(10)

df5 = to_df(num5)
df5.head(20)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504,81700.0
1,415,"Bavarian Collectables Imports, Co.",Donnermeyer,Michael,+49 89 61 08 9555,Hansastr. 15,None,Munich,None,80686,Germany,1504,77000.0
2,128,"Blauer See Auto, Co.",Keitel,Roland,+49 69 66 90 2555,Lyonerstr. 34,None,Frankfurt,None,60528,Germany,1504,59700.0
3,344,CAF Imports,Fernandez,Jesus,+34 913 728 555,Merchants House,27-30 Merchant's Quay,Madrid,None,28023,Spain,1702,59600.0
4,202,Canadian Gift Exchange Network,Tamuri,Yoshi,(604) 555-3392,1900 Oak St.,None,Vancouver,BC,V3F 2K1,Canada,1323,90300.0
5,189,"Clover Collections, Co.",Cassidy,Dean,+353 1862 1555,25 Maiden Lane,Floor No. 4,Dublin,None,2,Ireland,1504,69400.0
6,458,"Corrida Auto Replicas, Ltd",Sommer,Martín,(91) 555 22 82,"C/ Araquil, 67",None,Madrid,None,28023,Spain,1702,104600.0
7,385,Cruz & Sons Co.,Cruz,Arnold,+63 2 555 3587,15 McCallum Street,NatWest Center #13-03,Makati City,None,1227 MM,Philippines,1621,81500.0
8,171,Daedalus Designs Imports,Rancé,Martine,20.16.1555,"184, chaussée de Tournai",None,Lille,None,59000,France,1370,82900.0
9,145,Danish Wholesale Imports,Petersen,Jytte,31 12 3555,Vinbæltet 34,None,Kobenhavn,None,1734,Denmark,1401,83400.0


## Praktikum 5

1. Tampilkan semua data produk yang statusnya
sedang dikirim!

In [66]:
s = Session()
p = Product
o = Order
od = Orderdetail

num1 =  s.query(p.productCode, od.orderNumber, o.status).join(od.product).join(od.order).filter(o.status=='In Process')

df1 = to_df(num1)
df1

,productCode,orderNumber,status
0,S10_1949,10424,In Process
1,S10_4962,10425,In Process
2,S18_2238,10425,In Process
3,S18_3232,10425,In Process
4,S18_4933,10420,In Process
5,S24_1046,10420,In Process
6,S24_1444,10425,In Process
7,S24_1628,10420,In Process
8,S24_2766,10420,In Process
9,S24_2840,10425,In Process


2. Tampilkan pelanggan yang tinggal di negara
bagian yang sama dengan negara bagian tempat
karyawan yang melayaninya bekerja!

In [89]:
c = Customer
e = Employee
o = Office

num2 = s.query(c.customerName, c.state, 
               func.concat(e.firstName, e.lastName), o.state).join(c.employee).join(e.office).filter(c.state == o.state)

df2 = to_df(num2).rename(columns={'state':'customerState', 
                                  'concat_2':'employeeName', 
                                  'state_1':'employeeState'})
df2

,customerName,customerState,concat_1,employeeState
0,Mini Gifts Distributors Ltd.,CA,LeslieJennings,CA
1,Mini Wheels Co.,CA,LeslieJennings,CA
2,Technics Stores Inc.,CA,LeslieJennings,CA
3,Corporate Gift Ideas Co.,CA,LeslieJennings,CA
4,The Sharp Gifts Warehouse,CA,LeslieJennings,CA
5,Signal Collectibles Ltd.,CA,LeslieJennings,CA
6,Toys4GrownUps.com,CA,LeslieThompson,CA
7,Boards & Toys Co.,CA,LeslieThompson,CA
8,Collectable Mini Designs Co.,CA,LeslieThompson,CA
9,"Men 'R' US Retailers, Ltd.",CA,LeslieThompson,CA


3. Tampilkan pelanggan yang pernah melakukan
total pembayaran yang lebih besar dari 100.000

In [91]:
c = Customer
py = Payment

num3 = s.query(c.customerName, py.amount).join(py.customer).filter(py.amount>100000)

df3 = to_df(num3)
df3

,customerName,amount
0,Mini Gifts Distributors Ltd.,101244.59
1,Mini Gifts Distributors Ltd.,111654.40
2,Euro+ Shopping Channel,116208.40
3,Euro+ Shopping Channel,120166.58
4,"Dragon Souveniers, Ltd.",105743.00


4. Buat daftar semua pelanggan dan karyawan meskipun
mereka tidak pernah terlibat kegiatan transaksi.

In [113]:
import sqlalchemy
c = Customer
e = Employee

q1 = s.query(c.customerName).distinct()
q2 = s.query(func.concat(e.firstName, ' ', e.lastName)).distinct()
num4 = q1.union(q2)

df4 = to_df(num4)
df4

,customers_customerName
0,Atelier graphique
1,Signal Gift Stores
2,"Australian Collectors, Co."
3,La Rochelle Gifts
4,Baane Mini Imports
...,...
140,Peter Marsh
141,Tom King
142,Mami Nishi
143,Yoshimi Kato


In [116]:
q1 = s.query(c.customerName).distinct().count()
q2 = s.query(func.concat(e.firstName, ' ', e.lastName)).distinct().count()

print(q1+q2)

145


5. Tampilkan kolom tanggal pemesanan product dalam urutan menurun untuk
pesanan Ford Pickup Truck 1940.

In [27]:
p = Product
o = Order
od = Orderdetail

num1 =  s.query(p.productName, o.orderDate).join(od.product).join(od.order).filter(p.productName.contains('1940 Ford Pickup Truck')).order_by(desc(o.orderDate))

df1 = to_df(num1)
df1


,productName,orderDate
0,1940 Ford Pickup Truck,2005-05-31
1,1940 Ford Pickup Truck,2005-05-01
2,1940 Ford Pickup Truck,2005-03-09
3,1940 Ford Pickup Truck,2005-02-17
4,1940 Ford Pickup Truck,2005-01-20
5,1940 Ford Pickup Truck,2004-12-10
6,1940 Ford Pickup Truck,2004-11-29
7,1940 Ford Pickup Truck,2004-11-18
8,1940 Ford Pickup Truck,2004-11-04
9,1940 Ford Pickup Truck,2004-10-21


6. Tampilkan daftar nama produk yang dijual dengan harga kurang dari 80% dari
MSRP (Harga Eceran Yang Disarankan). (database classicmodels)

In [126]:
p = Product
od = Orderdetail
price = od.priceEach
msrp = p.MSRP

num6 = s.query(p.productName, price, msrp).join(od.product).filter(bp < 0.8*msrp)

df6 = to_df(num6)
df6

,productName,priceEach,MSRP
0,1969 Harley Davidson Ultimate Chopper,81.35,95.7
1,1969 Harley Davidson Ultimate Chopper,86.13,95.7
2,1969 Harley Davidson Ultimate Chopper,90.92,95.7
3,1969 Harley Davidson Ultimate Chopper,76.56,95.7
4,1969 Harley Davidson Ultimate Chopper,81.35,95.7
...,...,...,...
2991,Pont Yacht,54.05,54.6
2992,Pont Yacht,48.05,54.6
2993,Pont Yacht,52.42,54.6
2994,Pont Yacht,52.96,54.6


## Praktikum 6

1. Siapa saja customers yang status pesanannya tertahan karena melebihi batas kredit?

In [143]:
s = Session()
c = Customer
o = Order

num1 = s.query(c.customerName, o.comments, o.status).join(o.customer).filter(o.comments.contains('credit limit exceeded'), o.status == 'On Hold')
df1 = to_df(num1)
df1

,customerName,comments,status
0,Tekni Collectables Inc.,Customer credit limit exceeded. Will ship when...,On Hold
1,Gifts4AllAges.com,Customer credit limit exceeded. Will ship when...,On Hold
2,The Sharp Gifts Warehouse,Customer credit limit exceeded. Will ship when...,On Hold


2. Siapa saja customers yang berkomentar untuk menggunakan jasa pengantaran
DHL? Tampilkan nama mereka, status order mereka, serta komentar yang dimaksud.

In [142]:
c = Customer
o = Order

num2 = s.query(c.customerName, o.status, o.comments).join(o.customer).filter(o.comments.contains('DHL'))
df2 = to_df(num2)
df2

,customerName,status,comments
0,Euro+ Shopping Channel,Shipped,Customer requested that DHL is used for this s...
1,"La Corne D'abondance, Co.",Shipped,Customer requested that DHL is used for this s...
2,Gift Depot Inc.,Shipped,Customer requested that DHL is used for this s...
3,"Down Under Souveniers, Inc",Shipped,Customer requested that DHL is used for this s...
4,The Sharp Gifts Warehouse,Shipped,Customer requested that DHL is used for this s...
5,Microscale Inc.,Shipped,Customer requested that DHL is used for this s...


3. Tampilkan semua order mobil bermerk Ferrari pada database classicmodels!
Tampilkan pula nama customer, nama mobilnya, status order, serta tanggal
pengiriman yang berurut dari terkini. 

In [150]:
p = Product
o = Order
od = Orderdetail

num3 = s.query(p.productName, o).join(od.product).join(od.order).filter(p.productName.contains('Ferrari'))
df3 = to_df(num3)
df3

,productName,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,2001 Ferrari Enzo,10105,2003-02-11,2003-02-21,2003-02-12,Shipped,None,145
1,2001 Ferrari Enzo,10117,2003-04-16,2003-04-24,2003-04-17,Shipped,None,148
2,2001 Ferrari Enzo,10127,2003-06-03,2003-06-09,2003-06-06,Shipped,Customer requested special shippment. The inst...,151
3,2001 Ferrari Enzo,10142,2003-08-08,2003-08-16,2003-08-13,Shipped,None,124
4,2001 Ferrari Enzo,10153,2003-09-28,2003-10-05,2003-10-03,Shipped,None,141
...,...,...,...,...,...,...,...,...
75,1992 Ferrari 360 Spider red,10394,2005-03-15,2005-03-25,2005-03-19,Shipped,None,141
76,1992 Ferrari 360 Spider red,10405,2005-04-14,2005-04-24,2005-04-20,Shipped,None,209
77,1992 Ferrari 360 Spider red,10412,2005-05-03,2005-05-13,2005-05-05,Shipped,None,141
78,1992 Ferrari 360 Spider red,10419,2005-05-17,2005-05-28,2005-05-19,Shipped,None,382


## Praktikum 7

1. Menggunakan database classicmodels tampilkan nama pelanggan yang total
belanjanya melebihi kredit limit, serta tampilkan selisih total belanja dengan kredit
limit, kemudian urutkan berdasarkan selisih tersebut secara menurun.

In [9]:
s = Session()

2. Siapa customer yang total pembelian barangnya terbanyak (dihitung dari quantity
bukan price)

In [47]:
c = Customer
o = Order
od = Orderdetail

num2 = s.query(c.customerName, func.sum(od.quantityOrdered)).join(od.order).join(o.customer).group_by(c.customerNumber).order_by(desc(countOrder)).limit(1)
to_df(num2)

,customerName,sum_1
0,Euro+ Shopping Channel,9327.0


In [ ]:
data = s.query(o)